In [13]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import kerastuner as kt
from sklearn.model_selection import train_test_split
import seaborn as sns


def plot_curve(epochs, hist, list_of_metrics,name):
    """Plot a curve of one or more classification metrics vs. epoch."""  
    plt.figure()
    plt.xlabel("Epoch "+name)
    plt.ylabel("Value")

    for m in list_of_metrics:
        x = hist[m]
        plt.plot(epochs[1:], x[1:], label=m)
    plt.legend()
    
    
def create_model_optimizer(hp):
    model = tf.keras.models.Sequential()
    
   
    
    

    first_layer=hp.Int(str('1_units'), min_value = 32, max_value = 64, step = 8)
    model.add(tf.keras.layers.Dense(units = first_layer, activation = 'relu'))
    
    second_layer=hp.Int(str('2_units'), min_value = 64, max_value = 256, step = 16)
    model.add(tf.keras.layers.Dense(units = second_layer, activation = 'relu'))

    third_layer=hp.Int(str('3_units'), min_value = 128, max_value = 512, step = 128)
    model.add(tf.keras.layers.Dense(units = third_layer, activation = 'relu'))

    fourth_layer=hp.Int(str('4_units'), min_value = 512, max_value = 1024, step = 256)
    model.add(tf.keras.layers.Dense(units = fourth_layer, activation = 'relu'))

    fith_layer=hp.Int(str('5_units'), min_value = 128, max_value = 512, step = 32)
    model.add(tf.keras.layers.Dense(units = fith_layer, activation = 'relu'))


    six_layer=hp.Int(str('6_units'), min_value = 32, max_value = 128, step = 16)
    model.add(tf.keras.layers.Dense(units = six_layer, activation = 'relu'))



        
    hp_lr=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    hp_momentum=hp.Choice('momentum', values=[1e-2, 1e-1, 2e-1,5e-1])

    model.add(tf.keras.layers.Dense(units=2,name='Output', activation = 'relu'))                             
    model.compile(optimizer=tf.keras.optimizers.SGD(lr=hp_lr,momentum=hp_momentum),                                       
                loss=tf.keras.losses.MeanAbsoluteError(),
                metrics=[tf.keras.metrics.MeanAbsoluteError()])
    return model


def train_model(model,x_data, y_data, epochs, label_name,
                batch_size=None,shuffle=True):
    #features = {name:np.array(value) for name, value in dataset.items()}
    history = model.fit(x=x_data, y=y_data, batch_size=batch_size,
                      epochs=epochs, shuffle=shuffle,validation_split=0.2,
                       callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)])
  
    epochs = history.epoch
    hist = pd.DataFrame(history.history)
    return epochs, hist
    
    
#returns dataframe
def test_model(model,x_data, y_data ,label_name):
    evaluation=model.evaluate(x = x_data, y = y_data, batch_size=100)
    predicted = model.predict(x_data)
    df_test=pd.DataFrame(y_data,columns=[label_name])
   # print(predicted)
    df_predict=pd.DataFrame(predicted,columns=[label+"_pred" for label in label_name])
    return pd.concat([df_test,df_predict], axis=1)



In [14]:

all_labels_features=["delta","lambda","no_of_max","main_maximum"]
all_data=pd.read_csv("G:\OneDrive - bwedu\Master\Forschungspraktikum\Inga\pc-forschi\generated_Data/all_param_4_values_with_overlap.csv")
df_feature_labels=all_data[all_labels_features]


no_of_max_array=df_feature_labels["no_of_max"].to_numpy()
main_maximum_array=df_feature_labels["main_maximum"].to_numpy()

concat_label=df_feature_labels[["delta","lambda"]].to_numpy()
concat_feature=df_feature_labels[["main_maximum","no_of_max"]].to_numpy()


    
    
    
x_train, x_test,y_train,y_test = train_test_split( concat_feature, concat_label  ,test_size=0.20, random_state=42)


In [24]:
MAX_TRIALS = 20

EXECUTIONS_PER_TRIAL = 2

tuner = kt.RandomSearch(

    create_model_optimizer,

    objective='val_mean_absolute_error',

    max_trials=MAX_TRIALS,

    executions_per_trial=EXECUTIONS_PER_TRIAL,

    directory='delta_lambda_tuner',
    #overwrite = True,
    project_name='third Try',

    seed=14

)

tuner.search(x_train, y_train,batch_size=150, epochs = 40, validation_data = (x_test, y_test))


Epoch 1/40
350/350 [==============================] - ETA: 0s - loss: 0.3560 - mean_absolute_error: 0.356 - ETA: 0s - loss: 0.3403 - mean_absolute_error: 0.340 - ETA: 0s - loss: 0.3394 - mean_absolute_error: 0.339 - ETA: 0s - loss: 0.3355 - mean_absolute_error: 0.335 - ETA: 0s - loss: 0.3325 - mean_absolute_error: 0.332 - ETA: 0s - loss: 0.3310 - mean_absolute_error: 0.331 - ETA: 0s - loss: 0.3289 - mean_absolute_error: 0.328 - ETA: 0s - loss: 0.3267 - mean_absolute_error: 0.326 - ETA: 0s - loss: 0.3240 - mean_absolute_error: 0.324 - ETA: 0s - loss: 0.3217 - mean_absolute_error: 0.321 - ETA: 0s - loss: 0.3196 - mean_absolute_error: 0.319 - ETA: 0s - loss: 0.3174 - mean_absolute_error: 0.317 - ETA: 0s - loss: 0.3149 - mean_absolute_error: 0.314 - ETA: 0s - loss: 0.3128 - mean_absolute_error: 0.312 - ETA: 0s - loss: 0.3107 - mean_absolute_error: 0.310 - ETA: 0s - loss: 0.3084 - mean_absolute_error: 0.308 - ETA: 0s - loss: 0.3062 - mean_absolute_error: 0.306 - ETA: 0s - loss: 0.3041 - mea

350/350 [==============================] - ETA: 0s - loss: 0.2126 - mean_absolute_error: 0.212 - ETA: 0s - loss: 0.2057 - mean_absolute_error: 0.205 - ETA: 0s - loss: 0.2042 - mean_absolute_error: 0.204 - ETA: 0s - loss: 0.2053 - mean_absolute_error: 0.205 - ETA: 0s - loss: 0.2052 - mean_absolute_error: 0.205 - ETA: 0s - loss: 0.2050 - mean_absolute_error: 0.205 - ETA: 0s - loss: 0.2042 - mean_absolute_error: 0.204 - ETA: 0s - loss: 0.2043 - mean_absolute_error: 0.204 - ETA: 0s - loss: 0.2040 - mean_absolute_error: 0.204 - ETA: 0s - loss: 0.2041 - mean_absolute_error: 0.204 - ETA: 0s - loss: 0.2041 - mean_absolute_error: 0.204 - ETA: 0s - loss: 0.2039 - mean_absolute_error: 0.203 - ETA: 0s - loss: 0.2039 - mean_absolute_error: 0.203 - ETA: 0s - loss: 0.2039 - mean_absolute_error: 0.203 - ETA: 0s - loss: 0.2037 - mean_absolute_error: 0.203 - ETA: 0s - loss: 0.2038 - mean_absolute_error: 0.203 - ETA: 0s - loss: 0.2036 - mean_absolute_error: 0.203 - ETA: 0s - loss: 0.2036 - mean_absolute_

350/350 [==============================] - ETA: 0s - loss: 0.2003 - mean_absolute_error: 0.200 - ETA: 0s - loss: 0.1897 - mean_absolute_error: 0.189 - ETA: 0s - loss: 0.1895 - mean_absolute_error: 0.189 - ETA: 0s - loss: 0.1893 - mean_absolute_error: 0.189 - ETA: 0s - loss: 0.1897 - mean_absolute_error: 0.189 - ETA: 0s - loss: 0.1901 - mean_absolute_error: 0.190 - ETA: 0s - loss: 0.1902 - mean_absolute_error: 0.190 - ETA: 0s - loss: 0.1908 - mean_absolute_error: 0.190 - ETA: 0s - loss: 0.1908 - mean_absolute_error: 0.190 - ETA: 0s - loss: 0.1909 - mean_absolute_error: 0.190 - ETA: 0s - loss: 0.1909 - mean_absolute_error: 0.190 - ETA: 0s - loss: 0.1907 - mean_absolute_error: 0.190 - ETA: 0s - loss: 0.1907 - mean_absolute_error: 0.190 - ETA: 0s - loss: 0.1908 - mean_absolute_error: 0.190 - ETA: 0s - loss: 0.1907 - mean_absolute_error: 0.190 - ETA: 0s - loss: 0.1906 - mean_absolute_error: 0.190 - ETA: 0s - loss: 0.1905 - mean_absolute_error: 0.190 - ETA: 0s - loss: 0.1903 - mean_absolute_

350/350 [==============================] - ETA: 0s - loss: 0.1903 - mean_absolute_error: 0.190 - ETA: 0s - loss: 0.1838 - mean_absolute_error: 0.183 - ETA: 0s - loss: 0.1851 - mean_absolute_error: 0.185 - ETA: 0s - loss: 0.1859 - mean_absolute_error: 0.185 - ETA: 0s - loss: 0.1867 - mean_absolute_error: 0.186 - ETA: 0s - loss: 0.1865 - mean_absolute_error: 0.186 - ETA: 0s - loss: 0.1859 - mean_absolute_error: 0.185 - ETA: 0s - loss: 0.1858 - mean_absolute_error: 0.185 - ETA: 0s - loss: 0.1857 - mean_absolute_error: 0.185 - ETA: 0s - loss: 0.1854 - mean_absolute_error: 0.185 - ETA: 0s - loss: 0.1854 - mean_absolute_error: 0.185 - ETA: 0s - loss: 0.1852 - mean_absolute_error: 0.185 - ETA: 0s - loss: 0.1853 - mean_absolute_error: 0.185 - ETA: 0s - loss: 0.1850 - mean_absolute_error: 0.185 - ETA: 0s - loss: 0.1848 - mean_absolute_error: 0.184 - ETA: 0s - loss: 0.1848 - mean_absolute_error: 0.184 - ETA: 0s - loss: 0.1848 - mean_absolute_error: 0.184 - ETA: 0s - loss: 0.1847 - mean_absolute_

350/350 [==============================] - ETA: 0s - loss: 0.1798 - mean_absolute_error: 0.179 - ETA: 0s - loss: 0.1803 - mean_absolute_error: 0.180 - ETA: 0s - loss: 0.1807 - mean_absolute_error: 0.180 - ETA: 0s - loss: 0.1811 - mean_absolute_error: 0.181 - ETA: 0s - loss: 0.1817 - mean_absolute_error: 0.181 - ETA: 0s - loss: 0.1816 - mean_absolute_error: 0.181 - ETA: 0s - loss: 0.1816 - mean_absolute_error: 0.181 - ETA: 0s - loss: 0.1817 - mean_absolute_error: 0.181 - ETA: 0s - loss: 0.1816 - mean_absolute_error: 0.181 - ETA: 0s - loss: 0.1814 - mean_absolute_error: 0.181 - ETA: 0s - loss: 0.1812 - mean_absolute_error: 0.181 - ETA: 0s - loss: 0.1811 - mean_absolute_error: 0.181 - ETA: 0s - loss: 0.1810 - mean_absolute_error: 0.181 - ETA: 0s - loss: 0.1809 - mean_absolute_error: 0.180 - ETA: 0s - loss: 0.1811 - mean_absolute_error: 0.181 - ETA: 0s - loss: 0.1811 - mean_absolute_error: 0.181 - ETA: 0s - loss: 0.1810 - mean_absolute_error: 0.181 - ETA: 0s - loss: 0.1809 - mean_absolute_

...

350/350 [==============================] - ETA: 0s - loss: 0.1571 - mean_absolute_error: 0.157 - ETA: 2s - loss: 0.1670 - mean_absolute_error: 0.167 - ETA: 2s - loss: 0.1676 - mean_absolute_error: 0.167 - ETA: 2s - loss: 0.1684 - mean_absolute_error: 0.168 - ETA: 2s - loss: 0.1677 - mean_absolute_error: 0.167 - ETA: 2s - loss: 0.1673 - mean_absolute_error: 0.167 - ETA: 2s - loss: 0.1673 - mean_absolute_error: 0.167 - ETA: 2s - loss: 0.1680 - mean_absolute_error: 0.168 - ETA: 2s - loss: 0.1675 - mean_absolute_error: 0.167 - ETA: 2s - loss: 0.1679 - mean_absolute_error: 0.167 - ETA: 2s - loss: 0.1676 - mean_absolute_error: 0.167 - ETA: 2s - loss: 0.1677 - mean_absolute_error: 0.167 - ETA: 2s - loss: 0.1674 - mean_absolute_error: 0.167 - ETA: 1s - loss: 0.1672 - mean_absolute_error: 0.167 - ETA: 1s - loss: 0.1666 - mean_absolute_error: 0.166 - ETA: 1s - loss: 0.1666 - mean_absolute_error: 0.166 - ETA: 1s - loss: 0.1664 - mean_absolute_error: 0.166 - ETA: 1s - loss: 0.1664 - mean_absolute_

350/350 [==============================] - ETA: 0s - loss: 0.1614 - mean_absolute_error: 0.161 - ETA: 2s - loss: 0.1658 - mean_absolute_error: 0.165 - ETA: 2s - loss: 0.1662 - mean_absolute_error: 0.166 - ETA: 2s - loss: 0.1655 - mean_absolute_error: 0.165 - ETA: 2s - loss: 0.1662 - mean_absolute_error: 0.166 - ETA: 2s - loss: 0.1656 - mean_absolute_error: 0.165 - ETA: 2s - loss: 0.1654 - mean_absolute_error: 0.165 - ETA: 2s - loss: 0.1656 - mean_absolute_error: 0.165 - ETA: 2s - loss: 0.1655 - mean_absolute_error: 0.165 - ETA: 2s - loss: 0.1650 - mean_absolute_error: 0.165 - ETA: 2s - loss: 0.1647 - mean_absolute_error: 0.164 - ETA: 2s - loss: 0.1645 - mean_absolute_error: 0.164 - ETA: 2s - loss: 0.1645 - mean_absolute_error: 0.164 - ETA: 1s - loss: 0.1645 - mean_absolute_error: 0.164 - ETA: 1s - loss: 0.1645 - mean_absolute_error: 0.164 - ETA: 1s - loss: 0.1643 - mean_absolute_error: 0.164 - ETA: 1s - loss: 0.1643 - mean_absolute_error: 0.164 - ETA: 1s - loss: 0.1646 - mean_absolute_

350/350 [==============================] - ETA: 0s - loss: 0.1703 - mean_absolute_error: 0.170 - ETA: 2s - loss: 0.1631 - mean_absolute_error: 0.163 - ETA: 2s - loss: 0.1631 - mean_absolute_error: 0.163 - ETA: 2s - loss: 0.1627 - mean_absolute_error: 0.162 - ETA: 2s - loss: 0.1618 - mean_absolute_error: 0.161 - ETA: 2s - loss: 0.1617 - mean_absolute_error: 0.161 - ETA: 2s - loss: 0.1629 - mean_absolute_error: 0.162 - ETA: 2s - loss: 0.1634 - mean_absolute_error: 0.163 - ETA: 2s - loss: 0.1637 - mean_absolute_error: 0.163 - ETA: 2s - loss: 0.1637 - mean_absolute_error: 0.163 - ETA: 2s - loss: 0.1634 - mean_absolute_error: 0.163 - ETA: 2s - loss: 0.1636 - mean_absolute_error: 0.163 - ETA: 1s - loss: 0.1636 - mean_absolute_error: 0.163 - ETA: 1s - loss: 0.1637 - mean_absolute_error: 0.163 - ETA: 1s - loss: 0.1637 - mean_absolute_error: 0.163 - ETA: 1s - loss: 0.1636 - mean_absolute_error: 0.163 - ETA: 1s - loss: 0.1636 - mean_absolute_error: 0.163 - ETA: 1s - loss: 0.1638 - mean_absolute_

344/350 [============================>.] - ETA: 0s - loss: 0.1645 - mean_absolute_error: 0.164 - ETA: 2s - loss: 0.1628 - mean_absolute_error: 0.162 - ETA: 2s - loss: 0.1624 - mean_absolute_error: 0.162 - ETA: 2s - loss: 0.1615 - mean_absolute_error: 0.161 - ETA: 2s - loss: 0.1617 - mean_absolute_error: 0.161 - ETA: 2s - loss: 0.1614 - mean_absolute_error: 0.161 - ETA: 2s - loss: 0.1617 - mean_absolute_error: 0.161 - ETA: 2s - loss: 0.1620 - mean_absolute_error: 0.162 - ETA: 2s - loss: 0.1626 - mean_absolute_error: 0.162 - ETA: 2s - loss: 0.1624 - mean_absolute_error: 0.162 - ETA: 2s - loss: 0.1627 - mean_absolute_error: 0.162 - ETA: 2s - loss: 0.1631 - mean_absolute_error: 0.163 - ETA: 2s - loss: 0.1633 - mean_absolute_error: 0.163 - ETA: 1s - loss: 0.1636 - mean_absolute_error: 0.163 - ETA: 1s - loss: 0.1635 - mean_absolute_error: 0.163 - ETA: 1s - loss: 0.1632 - mean_absolute_error: 0.163 - ETA: 1s - loss: 0.1632 - mean_absolute_error: 0.163 - ETA: 1s - loss: 0.1634 - mean_absolute_

UnknownError: Failed to rename: delta_lambda_tuner\third Try\trial_72292e893e53f726be24c8df5fa15324\checkpoints\epoch_0\checkpoint_temp_aa3a2f0bab7f48b2accdb7853a533b3e/part-00000-of-00001.data-00000-of-00001 to: delta_lambda_tuner\third Try\trial_72292e893e53f726be24c8df5fa15324\checkpoints\epoch_0\checkpoint.data-00000-of-00001 : Access is denied.
; Input/output error [Op:MergeV2Checkpoints]

In [25]:
tuner.results_summary()


In [ ]:
best_model = tuner.get_best_models()[0]
all_label_list=["delta","lambda"]
delta_test_result=test_model(best_model,x_test,y_test,
                            all_label_list)


delta_test_result.head(50)